## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [57]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import AveragePooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LeakyReLU
from keras.layers import BatchNormalization
from keras.layers import GlobalAveragePooling2D
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


In [51]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=0)
        std = np.std(X_train, axis=0)
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [53]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

C:\Users\ben00\Anaconda3\envs\CV\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [70]:

classifier=Sequential()

#卷積組合 
classifier.add(Convolution2D(32, (3, 3), padding='same', input_shape = (32, 32, 3), activation='relu'))#32,3,3,input_shape=(32,32,3),activation='relu''
#classifier.add(LeakyReLU())
classifier.add(BatchNormalization())

'''自己決定MaxPooling2D放在哪裡'''
classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Convolution2D(32, (1, 1)))
classifier.add(LeakyReLU())
classifier.add(BatchNormalization())

#卷積組合
classifier.add(Convolution2D(32, (3, 3), padding='valid', dilation_rate=3, activation='relu'))
classifier.add(BatchNormalization())

classifier.add(Convolution2D(32, (1, 1), activation='relu'))
classifier.add(BatchNormalization())

#卷積組合
classifier.add(Convolution2D(32, (3, 3), padding='valid', activation='relu'))
#classifier.add(BatchNormalization())
classifier.add(Dropout(0.3))

#flatten
#classifier.add(Flatten())

#FC
#classifier.add(Dense(32, activation='relu')) #output_dim=100,activation=relu

#GAP
classifier.add(GlobalAveragePooling2D())

#輸出
classifier.add(Dense(output_dim=10,activation='sigmoid'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = keras.optimizers.Adam(lr=3e-4), loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.summary()
history = classifier.fit(x_train,y_train,batch_size=128,epochs=40, validation_data=(x_test, y_test))
#history = classifier.fit_generator(datagen.flow(x_train, y_train, batch_size=256), steps_per_epoch=len(x_train)/256, epochs=50)

C:\Users\ben00\Anaconda3\envs\CV\lib\site-packages\ipykernel_launcher.py:40: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=10)`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_164 (Conv2D)          (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization_151 (Bat (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_44 (MaxPooling (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_165 (Conv2D)          (None, 16, 16, 32)        1056      
_________________________________________________________________
leaky_re_lu_38 (LeakyReLU)   (None, 16, 16, 32)        0         
_________________________________________________________________
batch_normalization_152 (Bat (None, 16, 16, 32)        128       
_________________________________________________________________
conv2d_166 (Conv2D)          (None, 10, 10, 32)        9248      
__________

In [71]:
classifier.evaluate(x_test, y_test)

10000/10000 [==============================] - 2s 224us/step


[0.7402712723731995, 0.7454]

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [72]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[7.3045492e-05, 1.2814999e-06, 9.8924935e-03, 6.1580187e-01,
        2.4622229e-01, 7.0665866e-02, 9.9197626e-03, 3.9208174e-02,
        4.5180370e-04, 1.1511699e-05]], dtype=float32)